<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Grammar-and-corpus-loadinig" data-toc-modified-id="Grammar-and-corpus-loadinig-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Grammar and corpus loadinig</a></span></li><li><span><a href="#Recognizer" data-toc-modified-id="Recognizer-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Recognizer</a></span></li><li><span><a href="#Parser" data-toc-modified-id="Parser-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Parser</a></span><ul class="toc-item"><li><span><a href="#tree-extractor" data-toc-modified-id="tree-extractor-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>tree extractor</a></span></li></ul></li><li><span><a href="#report" data-toc-modified-id="report-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>report</a></span></li></ul></div>

In [1]:
# runtime: around 2min 30s

In [2]:
# import the necessary libraries
import nltk
from nltk.grammar import CFG
from nltk.tree import ImmutableTree

In [3]:
# define the path
grammar_file = "atis-grammar-cnf.cfg"
test_file = "atis-test-sentences.txt"

# Grammar and corpus loadinig

In [4]:
# load the grammar
grammar = nltk.data.load(grammar_file) 
# load test sentences
test_sentences = nltk.data.load(test_file) 
# extract test sentences
extracted_test_sentences = nltk.parse.util.extract_test_sentences(test_sentences) 

In [5]:
# create a parser and test and show not covered words
test_parser = nltk.parse.BottomUpChartParser(grammar)

# parse the test sentences
for sentence_chosen in extracted_test_sentences[:10]:
    try:
        test_parser.chart_parse(sentence_chosen[0])
    # handle exceptions
    except ValueError as error:
        # notifications
        print(f"Another sentence failed: {sentence_chosen[0]}, error is: {error}")


Another sentence failed: ['list', 'these', 'city', 'destinations', '.'], error is: Grammar does not cover some of the input words: "'destinations'".


To save time, I only test 10 in this case. However, after the test of whole dataset of several minutes, we can see that the grammar does not cover all the words in the test sentences, including the following words: destinations, count, buffalo and duration.

# Recognizer

In [6]:
def recognizer(grammar_input, sentences_input):
    len_sentence = len(sentences_input)
    
    # Check if the sentence is empty
    if len_sentence == 0:
        print("The sentence is empty.")
        return False

    # Create the chart of len_sentence * len_sentence
    chart = []
    for row in range(len_sentence):
        chart_row = []
        for column in range(len_sentence):
            chart_row.append(set())
        chart.append(chart_row)

    # Fill-in of the diagonal part as initialization
    for k in range(len_sentence):
        # Find the non-terminal symbols that can produce the word
        for product in grammar_input.productions(rhs=sentences_input[k]):
            chart[k][k].add(product.lhs())

    # Main loop: 
    # Current span
    for span_length in range(2, len_sentence + 1):  
        # Start position
        for start_position in range(0, len_sentence - span_length + 1): 
            # End position
            end_position = start_position + span_length - 1
            # Divide into two smaller spans
            for split_point in range(start_position, end_position):
                # Left small span
                for left_symbol in chart[start_position][split_point]:
                    # Right small span
                    for right_symbol in chart[split_point + 1][end_position]:
                        # Iterate through all productions to find matching ones
                        for product in grammar_input.productions():
                            if len(product.rhs()) == 2 and product.rhs() == (left_symbol, right_symbol):
                                chart[start_position][end_position].add(product.lhs())

    # If the start symbol is in the top right corner of chart or not.
    return grammar_input.start() in chart[0][len_sentence - 1]


In [7]:
# Test the ATIS test sentences for the recognizer
# around 20 seconds
count = 0
# too much time, so only test 2 sentences here
for idx in range(2):
    # Initialize the list
    test_sentence = extracted_test_sentences[idx][0]
    print(test_sentence)
    result = recognizer(grammar, test_sentence)
    print(f"sentence {idx}: {result}")
    # Count the number of grammatical sentences
    if result:
        count += 1
print(f"grammatical proportion: {count / len(extracted_test_sentences)}")


['prices', '.']
sentence 0: True
['show', 'availability', '.']
sentence 1: True
grammatical proportion: 0.02040816326530612


# Parser

In [9]:
# Define the parser
def parser(sentences_input, grammar_input):
    # grammar is a CFG object   
    
    #=============repeated codes as in recognizer start from here, just look at added codes below=================#
    len_sentence = len(sentences_input)
    
    # if the sentence is empty or not
    if len_sentence == 0:
        print("The sentence is empty.")
        return False
    
    # chart as in recognizer
    chart = []
    for row in range(len_sentence):
        chart_row = []
        for column in range(len_sentence):
            chart_row.append(set())
        chart.append(chart_row)
    
    # Fill the diagonal as in recognizer
    for k in range(len_sentence):
        # Find the non-terminal symbols that can produce the word
        for product in grammar_input.productions(rhs=sentences_input[k]):
            chart[k][k].add(product.lhs())
    #=======================================added codes start here==============================================#
    
    # creates back_pointers
    backpointer_list = []
    # add an empty dictionary for each
    for row in range(len_sentence):
        back_pointers_row = []
        for column in range(len_sentence):
            back_pointers_row.append({})
        backpointer_list.append(back_pointers_row)
    # Fill the diagonal
    for k in range(len_sentence):
        for product in grammar_input.productions(rhs=sentences_input[k]):
            backpointer_list[k][k][product.lhs()] = [sentences_input[k]]

    # Main CKY algorithm to fill the chart
    for span_length in range(2, len_sentence + 1):  # span_length is the width of the span
        # Start position
        for start_position in range(len_sentence - span_length + 1):
            end_position = start_position + span_length - 1
            # two smaller spans
            for split_point in range(start_position, end_position):
                # Left small span
                for left_symbol in chart[start_position][split_point]:
                    # Right small span
                    for right_symbol in chart[split_point + 1][end_position]:
                        # Check all grammar productions
                        for product in grammar_input.productions():
                            # Ensure the production is binary
                            if len(product.rhs()) == 2 and product.rhs() == (left_symbol, right_symbol):
                                # add to the chart
                                chart[start_position][end_position].add(product.lhs())
                                # add to the back_pointer_list
                                backpointer_list[start_position][end_position].setdefault(product.lhs(), []).append((left_symbol, right_symbol, split_point))

    return (grammar_input.start() in chart[0][len_sentence - 1], backpointer_list if grammar_input.start() in chart[0][len_sentence - 1] else None)


## tree extractor

In [10]:
# define Tree Extractor
def extract_trees(backpointer_list, start, end, start_symbol, memory=None):
    # Initialize memoization dict
    if memory is None:
        memory = {}

    # result for the current span already computed or not
    if (start, end, start_symbol) in memory:
        return memory[(start, end, start_symbol)]

    # diagonal of the chart
    if start == end:
        # Create a tree
        tree = ImmutableTree(start_symbol, [backpointer_list[start][end][start_symbol][0]])
        # Store
        memory[(start, end, start_symbol)] = {tree}
        return {tree}

    trees_set = set()

    # Iterate over all rules
    for left_non_terminal, right_non_terminal, split_point in backpointer_list[start][end][start_symbol]:
        # get all trees for the left and right spans
        left_trees = extract_trees(backpointer_list, start, split_point, left_non_terminal, memory)
        right_trees = extract_trees(backpointer_list, split_point + 1, end, right_non_terminal, memory)

        # Combine left and right
        for left_tree in left_trees:
            for right_tree in right_trees:
                trees_set.add(ImmutableTree(start_symbol, [left_tree, right_tree]))

    # Store the computed trees in the memoization dictionary
    memory[(start, end, start_symbol)] = trees_set
    return trees_set


In [11]:
# grammatical or not, and number of trees of all test sentences
# too much time consumed here if we try to test all sentences, so here only 3 are tested
# if you want to see all, check tree numbers.txt
# around 50 seconds
for sentence_chosen in extracted_test_sentences[:3]:
    try:
        # Parse the sentence
        grammatical, backpointer = parser(sentence_chosen[0], grammar)
        if not grammatical:
            # not grammatical so 0 tree.
            print(sentence_chosen[0], "\t", 0)
            continue
        # if grammatical, extract trees
        all_trees = extract_trees(backpointer, 0, len(sentence_chosen[0]) - 1, grammar.start())
        # Number of trees
        tree_num = len(all_trees) 
        # Print the number of trees
        print(sentence_chosen[0], "\t", tree_num)
    # notification of exceptions
    except Exception as error:
        print(f"Error happens. {sentence_chosen[0]}: {error}")
        continue
print("All over.")


['prices', '.'] 	 2
['show', 'availability', '.'] 	 3
['show', 'the', 'flights', '.'] 	 2
All over.


In [12]:
# tree of particular sentence
# 5 tree pngs stored in the folder of 5 trees
# around 1 min
index_test_sentence = 6
# sentence 6
sentence_chosen = extracted_test_sentences[index_test_sentence]
grammatical, backpointer = parser(sentence_chosen[0], grammar)
# if grammatical:
if grammatical:
    alltrees = extract_trees(backpointer, 0, len(sentence_chosen[0]) - 1, grammar.start())
    print("Sentence: ", sentence_chosen[0], len(all_trees))
    # Print multiple trees
    for tree in alltrees:
        tree = nltk.tree.Tree.fromstring(str(tree))
        tree.pretty_print()
        tree.draw()

Sentence:  ['list', 'saturday', 'flights', '.'] 2
                     SIGMA                         
         ______________|_____________               
      NP_NP                         KCJ            
   _____|______                ______|_______       
NP_NN       NOUN_NP        NOUN_NNS     pt_char_per
  |            |              |              |      
 list       saturday       flights           .     

                   SIGMA                       
    _________________|________                  
   |                         JWS               
   |                  ________|__________       
   |             RELCL_VBZ               |     
   |        _________|________           |      
NOUN_NN  NP_NP             VERB_VBZ pt_char_per
   |       |                  |          |      
  list  saturday           flights       .     

                 SIGMA                      
    _______________|_______                  
   |                      JKB               
   |     

# report

In [ ]:
# 5 trees pictures in folder of 5trees.
# number of trees of different senteces, in tree numbers.txt.